In [1]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import numpy as np
import pandas as pd
import os
import inspect
import requests
import json

In [2]:
site= "https://www.minoli.co.uk/tiles/page/1"
hdr = {'User-Agent': 'Mozilla/5.0'}
req = Request(site,headers=hdr)
page = urlopen(req)
soup = BeautifulSoup(page)


In [3]:
sitenum = 5000
i = 1
links = []
while i < sitenum:
    try:
        site= "https://www.minoli.co.uk/tiles/page/" + str(i)
        req = Request(site,headers=hdr)
        page = urlopen(req)
        soup = BeautifulSoup(page)
    except:
        print("Done")
        i = sitenum
    else:
        for link in soup.findAll('a'):
            links.append(link.get('href'))
        links = [i for i in links if i]
        substring = ["/tiles/"]
        matching = [s for s in links if any(xs in s for xs in substring)]
        print("Site done: " + str(i))
        i = i + 1

Site done 1
Site done 2
Site done 3
Site done 4
Site done 5
Site done 6
Site done 7
Site done 8
Site done 9
Site done 10
Site done 11
Site done 12
Site done 13
Site done 14
Site done 15
Site done 16
Site done 17
Site done 18
Site done 19
Site done 20
Site done 21
Site done 22
Site done 23
Site done 24
Site done 25
Site done 26
Site done 27
failure


In [4]:
matching = np.array(matching)
df = pd.DataFrame()
df['link']=pd.Series(matching)
df = df.drop_duplicates( keep='last')
df = df.reset_index(drop=True)

In [5]:
df_link = df[df["link"].str.contains("page")==False]

In [6]:
df = pd.DataFrame()
hdr = {'User-Agent': 'Mozilla/5.0'}
num = 1
for i in df_link['link']:
    site = i
    
    try:
        response = requests.get(site, headers=hdr)
        response.raise_for_status()
    except:
        print("failure")
        num = num + 1
    else:
        site = i
        page = requests.get(i).text
        soup = BeautifulSoup(page, "html.parser")
        divList = soup.find_all('div', attrs={"class" : "cell small-12 medium-6 large-3"})
        data = {}
        for div in divList:
            for k,v in div.attrs.items(): 
                if k not in ('class', 'id'):
                    k = k.split('-')[1]
                    data[k] = data.get(k, []) + [v]
        temp = pd.json_normalize(data)
        tmp = pd.DataFrame(temp)
        print(str(num) + ". Link done " + str(i))
        df = pd.concat([df, temp], axis = 0)
        num = num + 1

1. Link done https://www.minoli.co.uk/tiles/clay-outdoor-trust-2-2-2/
2. Link done https://www.minoli.co.uk/tiles/trust-outdoor-ivory-2/
3. Link done https://www.minoli.co.uk/tiles/citta-tortora/
4. Link done https://www.minoli.co.uk/tiles/clay-outdoor-trust-2-2/
5. Link done https://www.minoli.co.uk/tiles/twelvenoon-outdoor-daylight-2-2/
6. Link done https://www.minoli.co.uk/tiles/flux-outdoor-bone/
7. Link done https://www.minoli.co.uk/tiles/raw-outdoor-white-2-2-2/
8. Link done https://www.minoli.co.uk/tiles/clay-outdoor-trust/
9. Link done https://www.minoli.co.uk/tiles/bravestone-outdoor-coke/
10. Link done https://www.minoli.co.uk/tiles/norway-outdoor-farge/
11. Link done https://www.minoli.co.uk/tiles/laos-perla/
12. Link done https://www.minoli.co.uk/tiles/boost-outdoor-cream/
13. Link done https://www.minoli.co.uk/tiles/axis-outdoor-grey-timber-2/
14. Link done https://www.minoli.co.uk/tiles/lims-outdoor-desert/
15. Link done https://www.minoli.co.uk/tiles/etic-outdoor-rovere-

In [7]:
df.to_csv("result.csv", sep = ";")